In [1]:
import numpy as np
from aeon.distances import mpdist

In [19]:
y = np.array([[42, 23, 21, 55, 1, 19, 33, 34, 29, 19]])
print(y.shape)
if y.ndim == 2 and y.shape[0] > 1:
    raise ValueError("y must be a 1D array or shape (1,n)")
y = y.squeeze()
print(y.shape)

(1, 10)
(10,)


In [17]:
y

array([42, 23, 21, 55,  1, 19, 33, 34, 29, 19])

In [10]:
y = np.array([[11, 12, 13, 14],[7, 8, 9, 20],[1, 3, 4, 5]])
print(y.shape)
if y.ndim == 2 and y.shape[0] > 1:
    raise ValueError("y must be a 1D array or shape (1,n)")

(3, 4)


ValueError: y must be a 1D array or shape (1,n)

In [14]:
y = np.array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
print(y.shape)
if y.ndim == 2 and y.shape[0] > 1:
    raise ValueError("y must be a 1D array or shape (1,n)")
y = y.squeeze()
print(y.shape)

(10,)
(10,)


In [15]:
y

array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [20]:
y = np.array([11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [22]:
if y.ndim == 1:
    _y = y.reshape((1, y.shape[0]))
_y.shape

(1, 10)

In [2]:
x = np.array([1,2,3,5,9])
y = np.array([3,7,13,19])
m = 2

In [25]:
len(x)

5

In [3]:
n_x_subs = len(x) - m + 1
n_y_subs = len(y) - m + 1

In [27]:
x_mean = [np.mean(x[i : i + m]) for i in range(0, n_x_subs)]
x_mean

[1.5, 2.5, 4.0, 7.0]

In [4]:
# Compute the mean and standard deviation
x_mean = [np.mean(x[i : i + m]) for i in range(0, n_x_subs)]
x_std = [np.std(x[i : i + m]) for i in range(0, n_x_subs)]

y_mean = [np.mean(y[i : i + m]) for i in range(0, n_y_subs)]
y_std = [np.std(y[i : i + m]) for i in range(0, n_y_subs)]

In [5]:
print(x_mean)
print(x_std)
print(y_mean)
print(y_std)

[1.5, 2.5, 4.0, 7.0]
[0.5, 0.5, 1.0, 2.0]
[5.0, 10.0, 16.0]
[2.0, 3.0, 3.0]


In [6]:
type(x_mean)

list

In [7]:
x_mean = []
x_std = []
y_mean = []
y_std = []

for i in range(0, n_x_subs):
    x_subset = x[i:i+m]
    y_subset = y[i:i+m]
    
    x_mean.append(np.mean(x_subset))
    x_std.append(np.std(x_subset))
    
    y_mean.append(np.mean(y_subset))
    y_std.append(np.std(y_subset))

In [8]:
x_mean

[1.5, 2.5, 4.0, 7.0]

In [9]:
x_subset

array([5, 9])

In [10]:
t = x
t

array([1, 2, 3, 5, 9])

In [15]:
t_padded = np.pad(t, (0,len(t)))
t_padded

array([1, 2, 3, 5, 9, 0, 0, 0, 0, 0])

In [16]:
q = y[0:m]
q

array([3, 7])

In [17]:
q_reversed = np.flipud(q)
q_reversed

array([7, 3])

In [19]:
a = np.arange(8).reshape((2,4))
a

array([[0, 1, 2, 3],
       [4, 5, 6, 7]])

In [20]:
a_flipped = np.flipud(a)
a_flipped

array([[4, 5, 6, 7],
       [0, 1, 2, 3]])

In [29]:
def _sliding_dot_products(q, t):
    """
    Compute the sliding dot products between a query and a time series.

    Parameters
    ----------
        q: numpy.array
            Query.
        t: numpy.array
            Time series.

    Output
    ------
        dot_prod: numpy.array
            Sliding dot products between q and t.
    """
    q_len = len(q)
    t_len = len(t)

    # Reversing query and padding both query and time series
    t_padded = np.pad(t, (0, t_len))
    q_reversed = np.flipud(q)
    q_reversed_padded = np.pad(q_reversed, (0, 2 * t_len - q_len))
    
    # Applying FFT to both query and time series
    t_fft = np.fft.fft(t_padded)
    q_fft = np.fft.fft(q_reversed_padded)  # Flip the query and FFT it

    # Applying inverse FFT to obtain the convolution of the time series by
    # the query
    element_wise_mult = np.multiply(t_fft, q_fft)
    inverse_fft = np.fft.ifft(element_wise_mult)

    # Returns only the valid dot products from inverse_fft
    dot_prod = inverse_fft[q_len - 1 : t_len].real

    return dot_prod

In [30]:
prod = _sliding_dot_products(q,t)

In [31]:
prod

array([17., 27., 44., 78.])